In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/UCD/ECS-289G-DL
import sys
sys.path.append('/content/drive/MyDrive/UCD/ECS-289G-DL')

/content/drive/MyDrive/UCD/ECS-289G-DL


In [3]:
!pip install pytorch_lightning
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [4]:
from auto_encoder import AutoEncoder
from masked_dataset import CIFAR10DataModule
import torch
from torch import nn
from torch.nn import functional as F

In [5]:
batch_size = 1
cifar10 = CIFAR10DataModule(batch_size=batch_size)
cifar10.setup("nothing")
cifar10_train = cifar10.train_dataloader()

in_channel_size = 3
num_epochs = 10
model = AutoEncoder(in_channel_size=3)
model.eval()

device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

checkpoint = torch.load(
    "lightning_logs/version_1/checkpoints/last.ckpt", map_location=device)
model.load_state_dict(checkpoint["state_dict"])


  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


<All keys matched successfully>

In [6]:
original_size = 224*224

mseloss = nn.MSELoss(reduction="none")
weights = torch.zeros([original_size, original_size], dtype=torch.float16)
times_this_index_is_sampled = torch.zeros(original_size)

In [ ]:
for batch_idx, (masked, full, indices) in enumerate(cifar10_train):
    masked = masked.cuda()
    full = masked.cuda()
    # pass the sampled data to the model
    prediction = model(masked)
    # go through each image in the current batch
    for i in range(batch_size):
        # get loss per pixel across all channels
        loss = mseloss(prediction[i], full[i])
        single_loss = torch.sum(loss, dim=0)
        # get the indices that were in the sample
        idxs = indices[i]
        # add the losses to the index's loss tracker
        # iterate the number of samples seen
        weights[idxs] += single_loss.cpu()
        times_this_index_is_sampled[idxs] += 1

In [ ]:
torch.save(weights, "weights.pth")

# total loss / number of times seen
weights /= times_this_index_is_sampled

torch.save(weights, "divided.pth")

# normalize across rows
weights = F.normalize(weights, dim=0)

torch.save(weights, "embedding_tensor.pth")